### Comparing Models and Vectorization Strategies for Text Classification

This try-it focuses on weighing the positives and negatives of different estimators and vectorization strategies for a text classification problem.  In order to consider each of these components, you should make use of the `Pipeline` and `GridSearchCV` objects in scikitlearn to try different combinations of vectorizers with different estimators.  For each of these, you also want to use the `.cv_results_` to examine the time for the estimator to fit the data.

### The Data

The dataset below is from [kaggle]() and contains a dataset named the "ColBert Dataset" created for this [paper](https://arxiv.org/pdf/2004.12765.pdf).  You are to use the text column to classify whether or not the text was humorous.  It is loaded and displayed below.

**Note:** The original dataset contains 200K rows of data. It is best to try to use the full dtaset. If the original dataset is too large for your computer, please use the 'dataset-minimal.csv', which has been reduced to 100K.

In [101]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB

import pandas as pd

#NLP libraies
import nltk
from nltk.tokenize import word_tokenize,sent_tokenize
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer,TfidfTransformer
import nltk
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('punkt')

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import Pipeline, make_pipeline

import time
from datetime import datetime as dt
import numpy as np
     
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB

[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/jayakanthjayaram/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     /Users/jayakanthjayaram/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/jayakanthjayaram/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [102]:
df = pd.read_csv('text_data/dataset.csv')

In [103]:
df.head()

,text,humor
0,"Joe biden rules out 2020 bid: 'guys, i'm not r...",False
1,Watch: darvish gave hitter whiplash with slow ...,False
2,What do you call a turtle without its shell? d...,True
3,5 reasons the 2016 election feels so personal,False
4,"Pasco police shot mexican migrant from behind,...",False


#### Task


**Text preprocessing:** As a pre-processing step, perform both `stemming` and `lemmatizing` to normalize your text before classifying. For each technique use both the `CountVectorize`r and `TfidifVectorizer` and use options for stop words and max features to prepare the text data for your estimator.

**Classification:** Once you have prepared the text data with stemming lemmatizing techniques, consider `LogisticRegression`, `DecisionTreeClassifier`, and `MultinomialNB` as classification algorithms for the data. Compare their performance in terms of accuracy and speed.

Share the results of your best classifier in the form of a table with the best version of each estimator, a dictionary of the best parameters and the best score.

In [104]:
pd.DataFrame({'model': ['Logistic', 'Decision Tree', 'Bayes'], 
             'best_params': ['', '', ''],
             'best_score': ['', '', '']}).set_index('model')

,best_params,best_score
model,,
Logistic,,
Decision Tree,,
Bayes,,


In [105]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200000 entries, 0 to 199999
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   text    200000 non-null  object
 1   humor   200000 non-null  bool  
dtypes: bool(1), object(1)
memory usage: 1.7+ MB


# Prepocessing - stemming

In [106]:
def stemmer(text):
    stem = PorterStemmer()
    return ' '.join([stem.stem(w) for w in word_tokenize(text)])

def lemmatiz(text):
    lemma = WordNetLemmatizer()
    return ' '.join([lemma.lemmatize(w) for w in word_tokenize(text)])

In [107]:
stemmed_df = df.copy()
stemmed_df['text'] = df['text'].apply(stemmer)

In [108]:
stem_df = pd.DataFrame(stemmed_df)
stem_df.head()

,text,humor
0,"joe biden rule out 2020 bid : 'guy , i 'm not ...",False
1,watch : darvish gave hitter whiplash with slow...,False
2,what do you call a turtl without it shell ? de...,True
3,5 reason the 2016 elect feel so person,False
4,"pasco polic shot mexican migrant from behind ,...",False


In [109]:
lemma_df = df.copy()
lemma_df['text'] = df['text'].apply(lemmatiz)
lemma_df.head()

,text,humor
0,"Joe biden rule out 2020 bid : 'guys , i 'm not...",False
1,Watch : darvish gave hitter whiplash with slow...,False
2,What do you call a turtle without it shell ? d...,True
3,5 reason the 2016 election feel so personal,False
4,Pasco police shot mexican migrant from behind ...,False


In [110]:
X = df.drop('humor', axis = 1)
y = df['humor']

In [111]:
X_train, X_test, y_train, y_test = train_test_split(X['text'], y, test_size=0.30, random_state = 42)

In [ ]:
cvect = CountVectorizer()
cvect.fit_transform(X_train)
pd.DataFrame(dtm.toarray(), columns = cvect.get_feature_names()).tail()
pd.head(10)

In [113]:
pipe = Pipeline([('cvect', CountVectorizer()),
                ('lgr', MultinomialNB())])
pipe.fit(X_train, y_train)
test_acc = pipe.score(X_test, y_test)
params = {'cvect__max_features': [100, 300, 1000, 2000, 4000, 7000],
         'cvect__stop_words': ['english', None]}

grid = GridSearchCV(pipe, param_grid=params)
grid.fit(X_train, y_train)
acc = grid.score(X_test, y_test)
grid.best_params_, grid.best_score_

({'cvect__max_features': 7000, 'cvect__stop_words': None}, 0.9082642857142856)

In [114]:
params = {'cvect__max_features': [100, 300, 1000, 2000, 4000, 7000],
         'cvect__stop_words': ['english', None]}
grid = GridSearchCV(pipe, param_grid=params)
grid.fit(X_train, y_train)
acc = grid.score(X_test, y_test)

grid.best_params_, grid.best_score_

({'cvect__max_features': 7000, 'cvect__stop_words': None}, 0.9082642857142856)

In [115]:
X_s = stem_df.drop('humor', axis = 1)
y_s = stem_df['humor']

In [ ]:
start = dt.now()
# process stuff
grid_tf_mnb = GridSearchCV(pipe_stem_tfidf_mnb, param_grid=params_s_mnb)
grid_tf_mnb.fit(X_train_lr, y_train_lr)
acc_tf_mnb = grid_tf_mnb.score(X_test_lr, y_test_lr)

running_secs_nb = (dt.now() - start).seconds

grid_tf_mnb.best_params_, grid_tf_mnb.best_score_, test_acc_lr_mnb, running_secs_nb

In [ ]:
params_s_mnb1 = {'tfidf__max_features': [1000, 2000, 4000, 7000],
         'tfidf__stop_words': ['english', None], 
                'mnb__alpha': np.linspace(0.5, 1.5, 6),
                'mnb__fit_prior': [True, False],}

In [ ]:
start = dt.now()
# process stuff
grid_tf_mnb = GridSearchCV(pipe_stem_tfidf_mnb, param_grid=params_s_mnb1)
grid_tf_mnb.fit(X_train_lr, y_train_lr)
acc_tf_mnb = grid_tf_mnb.score(X_test_lr, y_test_lr)

running_secs_nb = (dt.now() - start).seconds

grid_tf_mnb.best_params_, grid_tf_mnb.best_score_, acc_tf_mnb, test_acc_lr_mnb, running_secs_nb

In [ ]:
pipe_stem_countvec_lr = Pipeline([('cvect', CountVectorizer()),
                ('lgr', LogisticRegression(max_iter=10000))])
pipe_stem_countvec_lr.fit(X_train_s, y_train_s)
test_acc_s_lr = pipe_stem_countvec_lr.score(X_test_s, y_test_s)

In [ ]:
params_s_lr = {'cvect__max_features': [100, 300, 1000, 2000, 4000, 7000],
         'cvect__stop_words': ['english', None]}

In [ ]:
start = dt.now()

grid_s_lr = GridSearchCV(pipe_stem_countvec_lr, param_grid=params_s_lr)
grid_s_lr.fit(X_train_s, y_train_s)

running_secs = (dt.now() - start).seconds

acc_s_lr = grid_s.score(X_test_s, y_test_s)

In [ ]:
grid_s_lr.best_params_, grid_s_lr.best_score_, running_secs

In [ ]:
X_lr = lemma_df.drop('humor', axis = 1)
y_lr = lemma_df['humor']

In [ ]:
X_train_lr, X_test_lr, y_train_lr, y_test_lr = train_test_split(X_lr['text'], y_lr, test_size=0.30, random_state = 42)

In [ ]:
pipe_stem_tfidf_lr = Pipeline([('tfidf', TfidfVectorizer()),
                ('lgr', LogisticRegression(max_iter=10000))])
pipe_stem_tfidf_lr.fit(X_train_lr, y_train_lr)
test_acc_s_lr = pipe_stem_tfidf_lr.score(X_test_lr, y_test_lr)

In [ ]:
params_s_lr = {'tfidf__max_features': [100, 300, 1000, 2000, 4000, 7000],
         'tfidf__stop_words': ['english', None]}

In [ ]:
start = dt.now()
# process stuff
grid_tf_lr = GridSearchCV(pipe_stem_tfidf_lr, param_grid=params_s_lr)
grid_tf_lr.fit(X_train_lr, y_train_lr)
acc_tf_lr = grid_tf_lr.score(X_test_lr, y_test_lr)

running_secs = (dt.now() - start).seconds

grid_tf_lr.best_params_, grid_tf_lr.best_score_, acc_tf_lr, running_secs

In [ ]:
pipe_stem_countvec_dt = Pipeline([('cvect', CountVectorizer()),
                ('dt', DecisionTreeClassifier())])
pipe_stem_countvec_dt.fit(X_train_s, y_train_s)
test_acc_s = pipe_stem_countvec_dt.score(X_test_s, y_test_s)

In [ ]:
params_s_dt = {'cvect__max_features': [100, 500, 1000, 2000],
         'cvect__stop_words': ['english', None],
              'dt__criterion': ['gini', 'entropy']}

In [ ]:
start = dt.now()

grid_s_dt = GridSearchCV(pipe_stem_countvec_dt, param_grid=params_s_dt, cv=5)
grid_s_dt.fit(X_train_s, y_train_s)
acc_s = grid_s_dt.score(X_test_s, y_test_s)

running_secs_dt = (dt.now() - start).seconds

grid_s_dt.best_params_, grid_s_dt.best_score_,grid_s_dt.cv_results_, running_secs_dt

In [ ]:
pipe_stem_tfidf_dt = Pipeline([('tfidf', TfidfVectorizer()),
                ('dt', DecisionTreeClassifier())])
pipe_stem_tfidf_dt.fit(X_train_lr, y_train_lr)
test_acc_s_lr = pipe_stem_tfidf_dt.score(X_test_lr, y_test_lr)

In [ ]:
params_s_lr = {'tfidf__max_features': [2000, 4000, 7000],
         'tfidf__stop_words': ['english', None],
              'dt__criterion': ['gini', 'entropy'],
              'dt__max_depth': [2, 3, 5, 10, 20]}

In [116]:
start = dt.now()

grid_tf_dt = GridSearchCV(pipe_stem_tfidf_dt, param_grid=params_s_lr)
grid_tf_dt.fit(X_train_s, y_train_s)
acc_s = grid_tf_dt.score(X_test_s, y_test_s)

running_secs_dt = (dt.now() - start).seconds

grid_tf_dt.best_params_, grid_tf_dt.best_score_,grid_tf_dt.cv_results_, running_secs_dt

NameError: name 'pipe_stem_tfidf_dt' is not defined

In [117]:
pd.DataFrame({'model': ['Logistic_stem', 'Logistic_lemm','DecisionTree_stem', 'DecisionTree_lemm',
                        'Bayes_stem', 'Bayes_lemm'], 
             'best_params': ['cvect__max_feature: 7000,cvect__stop_words:None',
                             'tfidf__max_features: 7000,tfidf__stop_words: None',
                             'cvect__max_features: 2000,dt__criterion: entropy',
                             'criterion: gini,max_depth:6,max_features:7000',
                             'cvect__max_features: 7000,cvect__stop_words: None',
                             'tfidf__max_features: 7000,tfidf__stop_words: None'],
              
             'best_score': ['0.92305', '0.91827', '0.86543', '0.78504', '0.90633', '0.90472'],
             'test_time_sec': ['153','132','1471','131','88','62']}).set_index('model') 

,best_params,best_score,test_time_sec
model,,,
Logistic_stem,"cvect__max_feature: 7000,cvect__stop_words:None",0.92305,153
Logistic_lemm,"tfidf__max_features: 7000,tfidf__stop_words: None",0.91827,132
DecisionTree_stem,"cvect__max_features: 2000,dt__criterion: entropy",0.86543,1471
DecisionTree_lemm,"criterion: gini,max_depth:6,max_features:7000",0.78504,131
Bayes_stem,"cvect__max_features: 7000,cvect__stop_words: None",0.90633,88
Bayes_lemm,"tfidf__max_features: 7000,tfidf__stop_words: None",0.90472,62
